### NLP - Natural Language Processing (Procesamiento de Lenguaje Natural)

En este notebook se hará el procesamiento de lenguaje natural (NLP) en los comentarios de los datasets de entrenamiento y prueba. Se utilizará la librería `nltk` para realizar el procesamiento de los comentarios.

Lo que se hará es:
1. Convertir los comentarios a minúsculas
2. Tokenizar los comentarios
3. Eliminar contenido que hace ruido -> URLs, menciones, hashtags, etc.
4. Reducir palabras extendidas (como "Holaaaaaaa" a "Hola")
5. Eliminar las palabras vacías (stopwords)
6. Lematizar las palabras
7. Eliminar los signos de puntuación
8. Eliminar los números
9. Eliminar los emojis
10. Eliminar los espacios en blanco

Luego una vez ya queden los tokens limpios, se entrenará un Word2Vec para obtener los vectores de palabras de los comentarios. (Promedio de los vectores de palabras de cada comentario)

In [6]:
# Instalación de librerías
%pip install -r '../requirements.txt'

Note: you may need to restart the kernel to use updated packages.


In [7]:
# Importación de librerías
# Expresiones regulares
import re
# Manejo de datos
import pandas as pd
# Procesamiento de texto
import nltk
from nltk.corpus import stopwords


In [8]:
# Carga de datos
# datos_entrenamiento = pd.read_csv('../2-traduccion-dataset/result/dataset-spanish-tweets-train.csv', delimiter=',', encoding='utf-8')
# datos_prueba = pd.read_csv('../2-traduccion-dataset/result/dataset-spanish-tweets-test.csv', delimiter=',', encoding='utf-8')
# datos_validacion = pd.read_csv('../2-traduccion-dataset/result/dataset-spanish-tweets-validation.csv', delimiter=',', encoding='utf-8')

In [9]:
# Definición de clase para el preprocesamiento de datos

class Preprocesamiento:
  stopwords_es = []
  stopwords_en = []

  def __init__(self):
    nltk.download('stopwords')
    nltk.download('punkt_tab')
    self.stopwords_es = stopwords.words('spanish')
    self.stopwords_en = stopwords.words('english')
    pass

  def preprocesar_texto(self, texto: str) -> list[str]:
    nuevo_texto = self.pasar_a_minusculas(texto)
    tokens = self.tokenizar_texto(nuevo_texto)
    tokens = self.eliminar_ruido(tokens)
    tokens = self.reducir_palabras_extendidas(tokens)
    tokens = self.eliminar_stopwords(tokens)

    return tokens

  # Paso 1: Convertir el texto a minúsculas
  def pasar_a_minusculas(self, texto: str) -> str:
    return texto.lower()

  # Paso 2: Tokenizar el texto
  # Ejemplo: "Hola, ¿cómo estás?" -> ["Hola,", "¿cómo", "estás?"]
  def tokenizar_texto(self, texto: str) -> list[str]:
    texto_tokenizado = texto.split(' ')
    texto_tokenizado = self.limpiar_tokens(texto_tokenizado)
    texto_tokenizado_limpio = self.eliminar_ruido(texto_tokenizado)
    texto_tokenizado_limpio = self.reducir_palabras_extendidas(texto_tokenizado_limpio)
    return texto_tokenizado_limpio

  # Paso 3: Eliminar ruido como URLs, menciones y hashtags
  # Ejemplo: ["Hola,"@sebastian, "http://www.google.com", "#InteligenciaArtificial", "¿cómo", "estás?"] -> ["Hola,", "¿cómo", "estás?"]
  def eliminar_ruido(self, tokens: list[str]) -> list[str]:
    # Eliminar URLs
    lista_limpia = []
    for token in tokens:
      if not token.startswith('http'):
        lista_limpia.append(token)
    # Eliminar Hashtags
    lista_limpia = [token for token in lista_limpia if not token.startswith('#')]
    # Eliminar menciones
    lista_limpia = [token for token in lista_limpia if not token.startswith('@')]
    return lista_limpia

  # Paso 4: Reducir palabras extendidas
  # Ejemplo: ["Holaaaa"] -> ["Hola"]
  def reducir_palabras_extendidas(self, tokens: list[str]) -> list[str]:
    for i in range(len(tokens)):
      tokens[i] = re.sub(r'(.)\1{2,}', r'\1\1', tokens[i])
    return tokens

  # Paso 5: Eliminar stopwords
  def eliminar_stopwords(self, tokens: list[str]) -> list[str]:
    tokens_limpio = [token for token in tokens if token not in self.stopwords_es]
    tokens_limpio = [token for token in tokens if token not in self.stopwords_en]
    return tokens_limpio

  # Helper 1: Limpiar tokens
  # Ejemplo: ["Hola,", "", "¿cómo", "estás?"] -> ["Hola,", "¿cómo", "estás?"]
  def limpiar_tokens(self, tokens: list[str]) -> list[str]:
    tokens_limpio = [token for token in tokens if token != '']
    return tokens_limpio

In [10]:
# Preprocesamiento de datos
preprocesamiento = Preprocesamiento()
comment = '¡Odioooooooooooooo el League of Legends porque  es  un  juego muy tóxico y la comunidad es muy mala! 😝 el link para descargarlo es https://www.leagueoflegends.com'
comment_preprocesado = preprocesamiento.preprocesar_texto(comment)
print(f'Comentario original: {comment}')
print(f'Comentario preprocesado: {comment_preprocesado}')

[nltk_data] Error loading stopwords: HTTP Error 500: Internal Server
[nltk_data]     Error


Comentario original: ¡Odioooooooooooooo el League of Legends porque  es  un  juego muy tóxico y la comunidad es muy mala! 😝 el link para descargarlo es https://www.leagueoflegends.com
Comentario preprocesado: ['¡odioo', 'el', 'league', 'legends', 'porque', 'es', 'un', 'juego', 'muy', 'tóxico', 'la', 'comunidad', 'es', 'muy', 'mala!', '😝', 'el', 'link', 'para', 'descargarlo', 'es']


[nltk_data] Error loading punkt_tab: HTTP Error 500: Internal Server
[nltk_data]     Error
